# Local Aggregations Module

In [1]:
def param_default():
    return {
        'model_name' : '/workspaces/code-rationales/data/codeparrot-small/checkpoints/checkpoint-29000', 
        'cache_dir': '/workspaces/code-rationales/datax/df_cache_dir',
        'max_generation_length': 50, 
        'number_experiments': 5, 
        'bootstrapping': 50
    }
prompts = [
    'def multiply_numbers(a*b):\n   print(\'This function multiply two numbers\')'
]

In [2]:
params = param_default()

## CORE

### Imports

In [3]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools
import json

pd.options.display.float_format = '{:.2f}'.format

In [4]:
from code_rationales.loader import download_grammars
from tree_sitter import Language, Parser
import code_rationales

In [5]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-07-19 20:44:02.844425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 20:44:03.002353: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import warnings
import importlib
from matplotlib import colors
import os

In [7]:
import sys
sys.path.insert(1, '/workspaces/code-rationales/sequential-rationales/huggingface')
from rationalization import rationalize_lm

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

### Taxonomies

In [9]:
#Programming Language Taxonomy
def pl_taxonomy_python() -> dict:
    return {
  "punctuation": ['{', '}', '[', ']', '(', ')','\"', ',', '.', '...', ';', ':'], 
  "exceptions": ['raise_statement','catch', 'try', 'finally', 'throw', 'throws', 'except'],
  "oop": ['def','class','instanceof','interface','private','protected','public','abstract','extends','package','this','implements','import','new','super'],
  "asserts": ['assert'],
  "types": ['tuple','set','list','pair','subscript','type','none','dictionary','integer','native','static','synchronized','transient','volatile','void','final','enum','byte','char','float','boolean','double','int','long','short','strictfp'],
  "conditionals": ['else', 'if', 'switch', 'case', 'default'],
  "loops": ['break', 'do', 'for', 'while', 'continue'],
  "operators": ['as','yield','is','@','in','and','or','not','**','slice','%','+','<','>','=','+','-','*','/','%','++','--','!','==','!=','>=','<=','&&','||','?',':','~','<<','>>','>>>','&','^','|','//'],
  "indentation": ['\n','\t'],
  "bool": ['true', 'false'], 
  "functional":['lambda','lambda_parameters'],
  "with" : ['with','with_item','with_statement','with_clause'], 
  "return" :['return'],
  "structural" : ['attribute','module', 'argument_list','parenthesized_expression','pattern_list','class_definition','function_definition','block'],
  "statements" : ['return_statement','break_statement','assignment','while_statement','expression_statement','assert_statement'],
  "expression": ['call','exec','async','ellipsis','unary_operator','binary_operator','as_pattern_target','boolean_operator','as_pattern','comparison_operator','conditional_expression','named_expression','not_operator','primary_expression','as_pattern'],
  "errors": ["ERROR"],
  "identifier":["identifier"],  
  "comment":["comment"],
  "string": ['string','interpolation','string_content','string_end','string_start','escape_sequence']  
}

### Model, Tokenizer Loading

In [10]:
model = AutoModelForCausalLM.from_pretrained(params['model_name'], cache_dir=params['cache_dir'])
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [11]:
tokenizer = AutoTokenizer.from_pretrained(params['model_name'])

### Local Dataset Creation

In [12]:
df_sampled_code = pd.DataFrame(prompts, columns=['prompt'])

In [13]:
df_sampled_code['input_ids'] = tokenizer(df_sampled_code['prompt'].tolist())['input_ids']

### Model Sampling Generation

In [14]:
def df_sampled_generation(
        df_sampled_code, 
        model,
        number_samples = 1,
        max_gen_tok = 100, 
        top_k = 0
    ):
    dict_generated_code = {i: [] for i in range(number_samples)}
    for idx_prompt, prompt in enumerate(df_sampled_code['prompt']):
        input = tokenizer([prompt], return_tensors="pt")
        input.to(model.device)
        outputs = model.generate(**input, do_sample=True,
                                 max_length=max_gen_tok,
                                 top_k=top_k, 
                                 num_return_sequences=number_samples, 
                                 pad_token_id=tokenizer.eos_token_id)
        for index, output in enumerate(outputs):
            dict_generated_code[index].append(output.tolist())
    df_temp = pd.DataFrame().from_dict(data=dict_generated_code) # DataFrame from Generation
    df_temp = pd.concat([df_sampled_code.reset_index(), df_temp ], axis=1) #Index before concating
    return df_temp

### Running Rationales

In [15]:
#If the model is not fine-tuned or compatible, it will rise an error
#This function works for one tensor of source token and one tensor of target tokens
def rationalize_model(model, tokenizer, input_ids, max_token_size: int, verbose=True):
    torch.cuda.empty_cache() #Cleaning Cache
    all_rationales, log = rationalize_lm(
        model = model,
        input_ids = input_ids[:max_token_size],
        tokenizer = tokenizer,
        verbose = verbose,
        max_steps=1024 #Max number of steps for greedy rationalization
    )
    return all_rationales, log 

In [16]:
def run_multiple_rational(
    model,
    tokenizer, 
    arr_target_tokens, 
    seq_id, #mapping sequence id
    max_token_size,
    verbose=True
):
    arr_log = []
    for index, val in enumerate(arr_target_tokens):
        all_rationales, log = rationalize_model(
            model=model, 
            tokenizer=tokenizer, 
            input_ids=val,
            max_token_size=max_token_size,
            verbose=False
        )
        arr_log.append(log)
    arr_code_rationales = [ log['rationalization'] for log in arr_log ] #extracting just rationalizations
    arr_from_sentence = [ list(np.full( len(val), seq_id[arr_i] )) #arr_i maps to the real sequence id
                            for arr_i, val in enumerate(arr_code_rationales)]
    arr_code_rationales = sum( arr_code_rationales, [] ) #flatting
    arr_from_sentence = sum( arr_from_sentence, [] ) #flatting
    return arr_code_rationales, arr_from_sentence

In [17]:
def pandas_rationales( arr_code_rationales, arr_from_sentence ):
    #Creating pandas_1 {p_rationale}
    rational = lambda list_log,typeset: [ (dict_tok['added_token_text'],round(dict_tok['true_token_prob'],6)) for dict_tok in list_log if dict_tok['from']==typeset]
    log = lambda log_row: [(log_dict['added_token_text'],log_dict['true_token_prob']) for log_dict in log_row] #Typeset

    log_position = lambda log_row: [log_dict['added_token_position'] for log_dict in log_row] #Position of the Rationale
    log_prediction = lambda log_row: [log_dict['true_token_prob'] for log_dict in log_row] #Rationale Prob

    p_rationale = pd.DataFrame()

    p_rationale['goal_token'] = [dict_token['goal_word'] for dict_token in arr_code_rationales]
    p_rationale['from_seq_id'] = arr_from_sentence

    p_rationale['typesets_tgt'] = [ log(log_row) for log_row in [dict_token['log'] for dict_token in arr_code_rationales]]
    
    p_rationale['rationale_pos_tgt'] = [ log_position(log_row) for log_row in [dict_token['log'] for dict_token in arr_code_rationales]]
    p_rationale['rationale_prob_tgt'] = [ log_prediction(log_row) for log_row in [dict_token['log'] for dict_token in arr_code_rationales]]


    return p_rationale

In [18]:
#Running Rationalization
def run_code_rational( 
        df_generated_input,
        tensor_size, #Control the size of the experiment
        experiment = '5',
        batch_size = 100, 
        model = model,
        max_token_size = 44,
        verbose = True 
    ):

    arr_rationals = []
    arr_from_seq = []

    for i in range( 0 , tensor_size , batch_size ):
        print('************************' + str(i) + '************************')
        t_generated_input = df_generated_input[experiment].values[i:i+batch_size]
        t_generated_input = [ torch.tensor(s).to(model.device) for s in t_generated_input]

        t_arr_rationals,t_arr_from_seq = run_multiple_rational(
            model = model,
            tokenizer = tokenizer,
            arr_target_tokens =  t_generated_input, 
            seq_id = list(range(i,i+batch_size)),
            max_token_size = max_token_size,
            verbose = verbose
        )

        arr_rationals = arr_rationals + t_arr_rationals
        arr_from_seq = arr_from_seq + t_arr_from_seq

        torch.cuda.empty_cache() #Cleaning Cache
        
    print("Experiment Finished: " + str(experiment))
    return pandas_rationales( arr_rationals, arr_from_seq )

In [19]:
def run_code_rational_all_set(exp, tensor_n = 100, BATCH = 10, max_token_size = 44): #When Tensor_n and batch differs then 'from_seq_id' is lost
    torch.cuda.empty_cache() #Cleaning Cache
    EXP = exp
    test_arr_rationals = run_code_rational( 
            df_generated_input = df_generated_input,
            tensor_size = tensor_n,
            experiment = EXP,
            batch_size = BATCH, 
            model = model,
            max_token_size = max_token_size,
            verbose = False 
        )
    #Saving process
    return test_arr_rationals


### AST Mapping

In [20]:
def unroll_node_types(
    nested_node_types: dict  # node_types from tree-sitter
) -> list: # list of node types
    def iterate_and_unroll_dict(nested_node_types: dict, all_node_types: set):
        for key, value in nested_node_types.items():
            if key == 'type' and type(value) == str:
                all_node_types.add(value)
            if type(value) == dict:
                iterate_and_unroll_dict(value, all_node_types)
            if type(value) == list:
                for element in value:
                    iterate_and_unroll_dict(element, all_node_types) 
    all_node_types = set()
    for dictionary in nested_node_types:
        iterate_and_unroll_dict(dictionary, all_node_types)
    all_node_types.add('ERROR')
    return list(all_node_types)

In [21]:
def create_parser(lang: str):
    # Grab the node types from the tree-sitter language
    language = Language(f"{code_rationales.__path__[0]}/grammars/tree-sitter-languages.so", lang)
    node_path = f"{code_rationales.__path__[0]}/grammars/tree-sitter-{lang}/src/node-types.json"
    with open(node_path) as f:
            node_types = json.load(f)
    node_types = unroll_node_types(node_types)
    # Create a parser for the language
    parser = Parser()
    parser.set_language(language)
    return parser, node_types

In [22]:
def traverse(
    node,       # tree-sitter node
) -> None:
    """Traverse in a recursive way, a tree-sitter node and append results to a list."""
    results = []
    def traverse_tree(node, results):
        if node.type == 'string':
            results.append(node)
            return
        for n in node.children:
            traverse_tree(n, results)
        if not node.children:
            results.append(node)
    traverse_tree(node, results)
    return results

In [23]:
def convert_to_offset(
    point,              #point to convert
    lines: list         #list of lines in the source code
    ):
        """Convert the point to an offset"""
        row, column = point
        chars_in_rows = sum(map(len, lines[:row])) + row
        chars_in_columns = len(lines[row][:column])
        offset = chars_in_rows + chars_in_columns
        return offset

In [24]:
def get_node_span(node, lines):
    """Get the span position of the node in the code string"""
    start_span = convert_to_offset(node.start_point, lines)
    end_span = convert_to_offset(node.end_point, lines)
    return start_span, end_span

In [25]:
def get_token_type(
    tok_span: tuple, # (start, end) position of a token in tokenizer
    nodes: list,     # list of tree-sitter nodes
    lines: list,     # list of lines in the code
) -> tuple: # (parent_type, token_type) of the token
    """Get the parent AST type and token AST type of a token."""
    node_spans = [get_node_span(node, lines) for node in nodes]
    for i, span in enumerate(node_spans):
        if (span[0] <= tok_span[0] and tok_span[0] < span[1]) or (span[0] < tok_span[1] and tok_span[1] <= span[1]):
            return nodes[i].parent.type, nodes[i].type

In [26]:
def get_token_nodes(
    tok_span: tuple, # (start, end) position of a token in tokenizer
    node,            # tree-sitter node
    lines: list,     # list of lines in the code
) -> list: 
    """Get all AST types for the given token span"""
    results = []
    def traverse_and_get_types(tok_span, node, lines, results) -> None:
        node_span = get_node_span(node, lines)
        if (node_span[0] <= tok_span[0] and tok_span[0] < node_span[1]) or (node_span[0] < tok_span[1] and tok_span[1] <= node_span[1]):
            results.append(node.type)
        for n in node.children:
            traverse_and_get_types(tok_span, n, lines, results)
    traverse_and_get_types(tok_span, node, lines, results)
    return results

### Rational Local Aggregates

In [27]:
calculate_left_span = lambda index, initial_token, df_rationals : len(str(initial_token) + ''.join(str(df_rationals['goal_token'][:index])))
calculate_right_span = lambda left_span, token : len(left_span) + len(token)

In [28]:
def clean_local_results(global_results):
    def clean_dictonary(result_dict):
        clean_dict = result_dict.copy()
        for key, value in result_dict.items():
            if not value:
                clean_dict.pop(key)
        return clean_dict
    for key, value in global_results.items():
        global_results[key] = clean_dictonary(value)
    return clean_dictonary(global_results)

In [29]:
def bootstrapping( np_data, np_func, size ):
    """Create a bootstrap sample given data and a function
    For instance, a bootstrap sample of means, or mediands. 
    The bootstrap replicates are a long as the original size
    we can choose any observation more than once (resampling with replacement:np.random.choice)
    """
    
    #Cleaning NaNs
    #np_data_clean = np_data[ np.logical_not( np.isnan(np_data) ) ] 
    
    #The size of the bootstrap replicate is as big as size
    #Creating the boostrap replicates as long as the orignal data size
    #This strategy might work as imputation 
    bootstrap_repl = [ np_func( np.random.choice( np_data, size=len(np_data) ) ) for i in range( size ) ]
    
    #logging.info("Covariate: " + cov) #Empirical Mean
    #logging.info("Empirical Mean: " + str(np.mean(np_data_clean))) #Empirical Mean
    #logging.info("Bootstrapped Mean: " + str( np.mean(bootstrap_repl) ) ) #Bootstrapped Mean
    
    return np.array( bootstrap_repl )

In [30]:
def bootstrap_samples_local_results(local_results: dict, size: int):
    local_results = local_results.copy()
    for target_type, target_value in local_results.items():
        for source_type, source_value in target_value.items():
            local_results[target_type][source_type] = bootstrapping(source_value, np.mean, size).tolist()
    return local_results

In [31]:
def aggregate_rationals(df_sample_experiments: list, parser, node_types: list, number_samples: int):
    experiments_aggregation = []
    for exp_idx, df_experiment in enumerate(df_sample_experiments):
        experiment_rational_result = df_experiment.reset_index()
        local_experiment_results = {goal_token : {node_type : [] for node_type in node_types} for goal_token in experiment_rational_result['goal_token']}
        initial_token = experiment_rational_result['typesets_tgt'][0][0][0]
        experiment_rational_result.insert(loc=experiment_rational_result.columns.get_loc('goal_token')+1, column='span', value=list(map(lambda tuple: (tuple[0],tuple[0]+len(tuple[1])),[(calculate_left_span(index, initial_token, experiment_rational_result), str(token)) for index, token in experiment_rational_result['goal_token'].items()])))
        target_code = experiment_rational_result['typesets_tgt'][0][0][0] + ''.join(str(experiment_rational_result['goal_token']))
        target_ast = parser.parse(bytes(target_code, 'utf8')).root_node
        for target_token_idx in range(len(experiment_rational_result['span'])):
            for rational_idx, rational_pos in enumerate(experiment_rational_result['rationale_pos_tgt'][target_token_idx]):
                if experiment_rational_result['rationale_pos_tgt'][target_token_idx][rational_idx] > 0: #rational 1 position.
                    try:
                        rational_node_types = get_token_nodes(experiment_rational_result['span'][experiment_rational_result['rationale_pos_tgt'][target_token_idx][rational_idx]-1], target_ast, target_code.split("\n"))
                        [local_experiment_results[experiment_rational_result['goal_token'][target_token_idx]][rational_node_type].append(experiment_rational_result['rationale_prob_tgt'][target_token_idx][rational_idx]) for rational_node_type in rational_node_types]
                    except Exception as e:
                        print('rational pos out of range')
        experiments_aggregation.append(clean_local_results(local_experiment_results))
    return experiments_aggregation

### Taxonomy Mapping 

In [32]:
def search_category_by_token(taxonomy_dict: dict, token_type: str):
    for key, value in taxonomy_dict.items():
        if token_type in value:
            return key
    return 'unknown'

In [33]:
def map_to_taxonomy(taxonomy_dict: dict, result_dict: dict):
    result_dict = result_dict.copy()
    mappings = {token: {category : [] for category in taxonomy_dict.keys()} for token in result_dict.keys()}
    mappings['unknown'] = {}
    for target_token, value in result_dict.items():
        for source_token, probs in value.items():
            mappings[target_token][search_category_by_token(taxonomy_dict, source_token)]+=probs
    return clean_local_results(mappings)

In [34]:
def convert_results_to_taxonomy(taxonomy_dict: dict, results:list):
    results = results.copy()
    for sample_idx, sample_result in enumerate(results):
        for exp_idx, experiment_result in enumerate(sample_result):
            results[sample_idx][exp_idx] = map_to_taxonomy(taxonomy_dict, experiment_result)
    return results


In [35]:
def bootstrap_local_results(local_results: list, size: int):
    local_results = local_results.copy()
    for sample_id, sample_result in enumerate(local_results):
        for exp_id, exp_result in enumerate(sample_result):
            for target_token, result_dict in exp_result.items():
                for rational, probs in result_dict.items():
                    local_results[sample_id][exp_id][target_token][rational] = bootstrapping(probs, np.mean, size)
    return local_results

## LOCAL EXPERIMENT

### Execute

In [36]:
### SAMPLING GENERATION 
df_generated_input = df_sampled_generation(
    df_sampled_code=df_sampled_code, 
    model=model, 
    number_samples=params['number_experiments'], 
    max_gen_tok=params['max_generation_length'])

In [37]:
### RATIONALES
experiment_results = []
for i in df_generated_input.columns[3:]: #Only Generated Sequences 
    experiment_result = run_code_rational_all_set(exp=i, tensor_n=df_generated_input.shape[0], BATCH=10, max_token_size=params['max_generation_length'])
    experiment_result['exp'] = i
    experiment_results.append(experiment_result)
df_experiment_results = pd.concat(experiment_results)

************************0************************
Experiment Finished: 0
************************0************************
Experiment Finished: 1
************************0************************
Experiment Finished: 2
************************0************************
Experiment Finished: 3
************************0************************
Experiment Finished: 4


In [38]:
### Gourping by sample
samples_experiments = [df_experiment_results[df_experiment_results['from_seq_id'] == sample_id] for sample_id in df_experiment_results['from_seq_id'].unique()]

In [39]:
### DEFINE PARSER
languages=['python', 'java']
download_grammars(languages)
parser, node_types = create_parser('python')

/usr/local/lib/python3.8/dist-packages/code_rationales/grammars
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
###AGGREGATE LOCAL RESULTS BY AST
results_by_ast = [aggregate_rationals([sample_experiments[sample_experiments['exp'] == exp_id] for exp_id in sample_experiments['exp'].unique()], parser, node_types, df_generated_input.shape[0]) for sample_experiments in samples_experiments]

In [41]:
###AGGREGATES BY TAXONOMY
taxonomy_results = convert_results_to_taxonomy(pl_taxonomy_python(), results_by_ast)

In [42]:
### BOOTSTRAPPING
taxonomy_results = bootstrap_local_results(taxonomy_results, params['bootstrapping'])
        

### Visualize - AST Aggregation 

In [ ]:
#results[<sample>][<experiment>][<token>] = grouped rationals dict
print(results_by_ast[0][0].keys())
print(results_by_ast[0][0]['numbers'].keys())
print(np.mean(results_by_ast[0][0]['numbers']['errors']))


### Visualize - Taxonomy Aggregation 

In [43]:
#results[<sample>][<experiment>][<token>] = grouped rationals dict
print(taxonomy_results[0][0].keys())
print(taxonomy_results[0][0]['numbers'].keys())
print(np.mean(taxonomy_results[0][0]['numbers']['errors']))


dict_keys([' multiply', '_', 'numbers', '(', 'a', '*', 'b', '):', '\n  ', ' print', "('", 'This', ' function', ' two', ' numbers', "')", 'MATCH', 'ES', ' TO', ' COMP', 'REPLY', ' full', 'number', ' =', " ''", ' analy', 'tical', ' run', 'times', ' 0', ' theta'])
dict_keys(['structural', 'statements', 'expression', 'errors', 'identifier'])
0.000702932826243341


In [49]:
taxonomy_results[0][0]['numbers'].keys()

dict_keys(['structural', 'statements', 'expression', 'errors', 'identifier'])